In [ ]:
import pandas as pd
from openai import OpenAI
from os import path
from glob import glob

# if any pack not installed, simply add a code below, type: !pip install pack_name, run it

In [ ]:
root = fr'D:\Dropbox\Documents\JHt\automation'
# set the root directory to your local folder

In [ ]:
"""
Copy the company list GPT provided to you to a text file, seperate by "; " (you can ask GPT to provide the list in this format)
Save it to the folder
You can save them to multiple text files as long as they are in the same folder.

"""

company_txts = glob(root + '*.txt')
# sort the list by name
company_txts.sort()

In [ ]:
company_txts

In [ ]:
companies = []
for txt in company_txts:
    with open(txt, 'r') as f:
        lines = f.readlines()
        companies_temp = [x.strip() for x in lines[0].split(';')]
        companies.extend(companies_temp)

In [ ]:
df = pd.DataFrame(companies, columns=['company_name', 'url', 'excel_hyperlink', 'status'])
df.fillna('', inplace=True)
df.sort_values('company_name', inplace=True)

In [ ]:
df.drop_duplicates(subset=['company_name'], keep='first', inplace=True)

In [ ]:
import urllib.parse
df['url'] = df['company_name'].apply(lambda x: f'https://www.google.com/search?q={urllib.parse.quote(x)}+careers')

In [ ]:
df['excel_hyperlink'] = df['url'].apply(lambda x: f'=HYPERLINK("{x}")')

In [ ]:
df.to_excel(root + 'companies.xlsx', index=False)

If you want to ask ChatGPT questions programatically, follow steps after

In [ ]:
import time

In [ ]:
df = pd.read_excel(root + 'companies.xlsx')

In [ ]:
df[['desired_job_probability', 'desired_job_reasoning']] = ''

In [ ]:
api_key = 'sk-_____________________' # your openai api key

In [ ]:
client = OpenAI(api_key=api_key)

for i, row in df.iterrows():
    company_name = row['company_name']
    query = f'''I am interested in working as a trader. Here, I will provide you with a company name. Use your own judgement and knowledge base,\
        tell me the possibility that this company has a trader position or not. "1" means the company has to have a trader position, otherwise it should not be able \
        to continously operate; "0" means this company surely does not have a trader position, its operation has nothing to do with trading. \
        Please provide me with the reasoning as well: for example, your knowledge base already owns the information that this company has a trader position, or the nature of \
        this company's operation requires a trader position, etc.
        
        Note, keep your judgement consistent and base solely on your knowledge base. I will show several companies' names twice, if you provide completely different answers, \
        I will discard your answers.

        Here are some examples:
        Company Name: HSBC Asset Management; Score: 1; Reasoning: My knowledge base confirms HSBC has asset management division, which requires a trader position.
        Company Name: BMO Capital Markets; Score: 1; Reasoning: My knowledge base confirms BMO Capital Markets includes its trading division.
        Company Name: Ewing Morris & Co Investment Partners Ltd. Score: 0.2; Reasoning: My knowledge base confirms Ewing Morris & Co Investment Partners Ltd. is a private equity firm, which is not likely to require a trader position.

        Now, it is your turn. Company Name: {company_name}; Score and Reasoning: '''

    model = "gpt-4o-mini"
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        model=model,
        temperature=0.1
    )
    answer = chat_completion.choices[0].message.content
        
    df.at[i, 'desired_job_reasoning'] = answer
    print(f'Completed {i} companies.')
    time.sleep(1)
    
    if i % 10 == 0:
        df.to_csv(path.join(root, 'df_temp.csv'), index=False)

In [ ]:
df.to_csv(path.join(root, 'df_temp.csv'), index=False)

In [ ]:
import re

pattern = r'Score: ([0-9.]+); Reasoning: (.+)'

df['desired_job_probability'] = df['desired_job_reasoning'].apply(lambda x: float(re.search(pattern, x).group(1)))
df['desired_job_reasoning'] = df['desired_job_reasoning'].apply(lambda x: re.search(pattern, x).group(2))

In [ ]:
df['desired_job_probability'].describe()

In [ ]:
df.to_excel(root + 'companies.xlsx', index=False)